In [3]:
chk_path ='/intentidentification/checkpoints_general_request/best-checkpoint-v6.ckpt'

In [4]:
from transformers_interpret import SequenceClassificationExplainer

from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
import torch.nn as nn
import pytorch_lightning as pl 
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

from transformers import (
    BertForMaskedLM,
    AutoTokenizer,
    AutoConfig,
    BertModel, 
    BertConfig, 
    BertTokenizer, 
    BertForSequenceClassification
)

LABEL_COLUMNS = ['general', 'request']
def print_model_info(model):
    # Get all of the model's parameters as a list of tuples.
    params = list(model.named_parameters())

    print('The BERT model has {:} different named parameters.\n'.format(len(params)))

    print('==== Embedding Layer ====\n')

    for p in params[0:5]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== First Transformer ====\n')

    for p in params[5:21]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== Output Layer ====\n')

    for p in params[-4:]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

def tokenize_sentences(sentences):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # encode_plus will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the [CLS] token to the start.
        #   (3) Append the [SEP] token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to max_length
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 320,           # Pad & truncate all sentences.
                            padding='max_length',
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids,attention_masks        

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

checkpoint = torch.load(chk_path, map_location=torch.device('cpu'))

# config = BertConfig.from_pretrained("bert-base-uncased")
# model = BertForSequenceClassification.from_pretrained(
#     "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = len(LABEL_COLUMNS),
#     state_dict=checkpoint
# )

model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2
)

print_model_info(model)

#load specified model state

# print(checkpoint["state_dict"].get('bert.embeddings.word_embeddings.num_embeddings'))
model.load_state_dict(checkpoint["state_dict"])
model.eval()
model.to(device)

Loading BERT tokenizer...


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
while(True):
    string = input("How may I help you? Question: ")
    #tokenize inputted sentence to be compatible with BERT inputs
    token_ids,attention_masks = tokenize_sentences([string])
    #get a tensor containing probabilities of inputted sentence being irrelevant or relevant
    model_outputs = (model(token_ids.to(device), token_type_ids=None, attention_mask=attention_masks.to(device)))
    softmax_layer = torch.nn.Softmax()
    result = softmax_layer(model_outputs[0])
    #identify which output node has higher probability and what that probability is
    prediction = torch.argmax(result).item()
    confidence = torch.max(result).item()
    print("I see. The problem is related to: " + LABEL_COLUMNS[prediction].replace("_", ' ') + " with {:.2f}% confident".format(confidence*100))
    print("-"*80)

How may I help you? Question: How are you?
I see. The problem is related to: general with 100.00% confident
--------------------------------------------------------------------------------
How may I help you? Question: What is your recommendations on hotel?
I see. The problem is related to: request with 100.00% confident
--------------------------------------------------------------------------------
How may I help you? Question: Life has been great.
I see. The problem is related to: general with 100.00% confident
--------------------------------------------------------------------------------
How may I help you? Question: I would like to know more about joint account.
I see. The problem is related to: request with 100.00% confident
--------------------------------------------------------------------------------
How may I help you? Question: When will my package be delivered.
I see. The problem is related to: request with 100.00% confident
----------------------------------------------